In [1]:
!pip install -q tensorflow_datasets

In [2]:
import tensorflow as tf 
import tensorflow_datasets as tfds
imdb,info=tfds.load('imdb_reviews',with_info=True,as_supervised=True)

0, 1979.05 examples/s]
Generating unsupervised examples...: 100%|█████████▉| 49817/50000 [00:29<00:00, 2324.67 examples/s]
                                                                                                   
Shuffling imdb_reviews-unsupervised.tfrecord...:   0%|          | 0/50000 [00:00<?, ? examples/s]
Shuffling imdb_reviews-unsupervised.tfrecord...:   7%|▋         | 3507/50000 [00:00<00:01, 35067.44 examples/s]
Shuffling imdb_reviews-unsupervised.tfrecord...:  89%|████████▉ | 44592/50000 [00:00<00:00, 166895.61 examples/s]
Dataset imdb_reviews downloaded and prepared to /home/shankar/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [8]:
import numpy as np 
train_data,test_data=imdb['train'],imdb['test']
train_sentences=[]
test_sentences=[]
train_labels=[]
test_labels=[]
for sentence,label in train_data:
    train_sentences.append(sentence.numpy().decode('utf8'))
    train_labels.append(label.numpy())
for sentence,label in test_data:
    test_sentences.append(sentence.numpy().decode('utf8'))
    test_labels.append(label.numpy())
train_labels=np.array(train_labels)
test_labels=np.array(test_labels)

In [18]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
voc=10000
embeddin_dim=16
max_length=120
truncate_type='post'
paddin_type='post'
oov_token='<oov>'
tokenizer=Tokenizer(num_words=voc,oov_token=oov_token)
tokenizer.fit_on_texts(train_sentences)
train_sequences=tokenizer.texts_to_sequences(train_sentences)
train_padded=pad_sequences(sequences=train_sequences,maxlen=120,padding=paddin_type,truncating=truncate_type)
test_sequences=tokenizer.texts_to_sequences(test_sentences)
test_padded=pad_sequences(sequences=test_sequences,maxlen=120,padding=paddin_type,truncating=truncate_type)

In [19]:
model=tf.keras.Sequential([tf.keras.layers.Embedding(voc,embeddin_dim,input_length=max_length),
                           tf.keras.layers.Flatten(),
                           tf.keras.layers.Dense(units=6,activation='relu'),
                           tf.keras.layers.Dense(units=1,activation='sigmoid')
                           ])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])            
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.fit(x=train_padded,y=train_labels,epochs=10,validation_data=(test_padded,test_labels))

Epoch 1/10
782/782 [==============================] - 10s 12ms/step - loss: 0.6093 - accuracy: 0.6208 - val_loss: 0.3853 - val_accuracy: 0.8252
Epoch 2/10
782/782 [==============================] - 13s 16ms/step - loss: 0.2401 - accuracy: 0.9078 - val_loss: 0.4069 - val_accuracy: 0.8230
Epoch 3/10
782/782 [==============================] - 10s 13ms/step - loss: 0.0932 - accuracy: 0.9784 - val_loss: 0.5133 - val_accuracy: 0.8059
Epoch 4/10
782/782 [==============================] - 11s 15ms/step - loss: 0.0233 - accuracy: 0.9973 - val_loss: 0.6053 - val_accuracy: 0.8094
Epoch 5/10
782/782 [==============================] - 15s 19ms/step - loss: 0.0072 - accuracy: 0.9995 - val_loss: 0.6708 - val_accuracy: 0.8054
Epoch 6/10
782/782 [==============================] - 14s 18ms/step - loss: 0.0026 - accuracy: 0.9999 - val_loss: 0.7321 - val_accuracy: 0.8071
Epoch 7/10
782/782 [==============================] - 12s 16ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.7868 - val_accuracy:

In [57]:
sentence=['This impressive remake of the 1933 classic benefits from state-of-the-art special effects but couldve used more old-fashioned restraint']
seq=tokenizer.texts_to_sequences(sentence)
padded=pad_sequences(maxlen=max_length,sequences=seq,padding=paddin_type,truncating=truncate_type)
pred=model.predict_classes(padded)
if pred[0][0]==1:
    print('positive review')
else:
    print('negative review')

positive review
